# Code for Group Work 612 Project
Group 4  BAN 612 Fall Semester 2018 Section 1

Ria Khanna

Philip Cole-Regis 

Xinyang Sherry Li

Haomeng Cai

## Retrieve Raw Data
Greenhouse is an applicant tracking system and recruiting software designed to optimize your entire recruiting process.
By knowing the name of a company that knows the uses this sevice, we can make use of Greenhouse's public api to get all the information associated with said company's job postings.


Note: This is just an example of the code used to access a few API urls. Actual dataset for this project uses many more companies.

In [22]:
#Define function 'greenhousejobs' to retrieve raw data

import pandas as pd
from pandas.io.json import json_normalize
import requests

def greenhousejobs(companies,jobdf):
    #INPUTS: List of company names and dataframe to append extracted job information
    for company in companies:
        #GET RELEVANT JSON DATA
        jsondata = requests.get("https://boards-api.greenhouse.io/v1/boards/"+company+"/jobs/").json()['jobs']
        #PICK OUT KEY FIELDS
        data = json_normalize(jsondata)[['title','location.name','absolute_url']]
        jobids = json_normalize(jsondata)['id']
        data.columns = ['title','name','url']  #rename columns
        data["company"] = pd.Series(company,index=data.index)
        #FOR JOB DESCRIPTIONS, GO TO EACH POSTING'S SPECIFIC PAGE. SAVE DESCRIPTION AS TEXT DATA, TO BE SEARCHED FOR KEYWORDS LATER. 
        desc = pd.Series(index=data.index)
        for index, row in data.iterrows():
            apiurl = "https://boards-api.greenhouse.io/v1/boards/" + company+ "/jobs/" + str(jobids[index])
            desc[index] = requests.get(apiurl).json()['content']
        data["description"] = desc
        #APPEND THIS COMPANY'S JOBS TO OVERALL DATAFRAME
        jobdf = jobdf.append(data,ignore_index=True)
    return jobdf

In [23]:
#Example data retrieval using 5 companies::
companies = ['khanacademy','twilio']
jobdf = pd.DataFrame([])
jobdf = greenhousejobs(companies,jobdf)
print jobdf.tail()

morecomapanies = ['github','pinterest','zume']
jobdf = greenhousejobs(morecomapanies,jobdf)
#jobdf.tail()

                                                 title               name  \
176                         Technical Support Engineer   Tallinn, Estonia   
177  Technical Training Enablement Manager, Enterpr...  San Francisco, CA   
178                    Technology Risk Program Manager  San Francisco, CA   
179               Vendor Security Risk Program Manager  San Francisco, CA   
180                                    VP of Messaging  San Francisco, CA   

                                                  url company  \
176  https://boards.greenhouse.io/twilio/jobs/1396465  twilio   
177  https://boards.greenhouse.io/twilio/jobs/1309645  twilio   
178  https://boards.greenhouse.io/twilio/jobs/1437440  twilio   
179  https://boards.greenhouse.io/twilio/jobs/1284747  twilio   
180   https://boards.greenhouse.io/twilio/jobs/854310  twilio   

                                           description  
176  &lt;p&gt;&lt;strong&gt;&lt;em&gt;Because you b...  
177  &lt;p&gt;&lt;strong&gt;Tech

## Filter and Reindex Raw Data
Data is filtered by job title and then reindexed so each record has a unique index

In [24]:
import numpy as np
filtereddata =jobdf[jobdf.title.str.contains('|'.join( ['Data', 'Quantative'])) | \
                     np.logical_and.reduce([jobdf.title.str.contains(word) for word in ['Business','Analyst']]) | \
                    np.logical_and.reduce([jobdf.title.str.contains(word) for word in ['Business','Intelligence']])]
filtereddata.drop_duplicates 
filtereddata.head()

,title,name,url,company,description
30,Data Architect,"San Francisco, CA",https://boards.greenhouse.io/twilio/jobs/1427475,twilio,&lt;p&gt; &lt;/p&gt;\n&lt;p&gt;&lt;strong&gt;&...
31,Data Engineer,"San Francisco, CA",https://boards.greenhouse.io/twilio/jobs/1289826,twilio,&lt;div&gt;\n&lt;p&gt; &lt;strong&gt;&lt;em&gt...
32,Data Science Intern - Summer 2019,"San Francisco, CA",https://boards.greenhouse.io/twilio/jobs/1302259,twilio,&lt;p&gt;&lt;strong&gt;&lt;em&gt;Because you b...
33,Data Scientist - New Grad - Understand,"San Francisco, CA",https://boards.greenhouse.io/twilio/jobs/1302257,twilio,&lt;p&gt;&lt;strong&gt;&lt;em&gt;Because you b...
34,Data Scientist - New Grad - Voice,"San Francisco, CA",https://boards.greenhouse.io/twilio/jobs/1305990,twilio,&lt;p&gt;&lt;strong&gt;&lt;em&gt;Because you b...


In [25]:
import numpy as np

filtereddata =jobdf[jobdf.title.str.contains('|'.join( ['Data', 'Quantative'])) | \
                     np.logical_and.reduce([jobdf.title.str.contains(word) for word in ['Business','Analyst']]) | \
                    np.logical_and.reduce([jobdf.title.str.contains(word) for word in ['Business','Intelligence']])]

filtereddata.drop_duplicates #drop repeat records
filtereddata.reset_index(drop=True,inplace=True)   #reindex

data = filtereddata  #rename dataframe to simply be called 'data' for ease later

# if data is saved is in two different dataframes, can combine them
#filtereddata= filtereddata.append(SomeOtherFilteredData,ignore_index=True)

#filtereddata.head()
filtereddata.tail()


,title,name,url,company,description
29,"Software Engineer, Big Data Platform","San Francisco, CA",https://careers.pinterest.com/careers/details/...,pinterest,&lt;p&gt;&lt;span style=&quot;font-weight: 400...
30,Business Analyst,"San Francisco, CA",https://boards.greenhouse.io/zume/jobs/4091417002,zume,&lt;h3&gt;&lt;strong data-redactor-tag=&quot;s...
31,Business Analyst,"Greenwich, CT or New York, NY",https://boards.greenhouse.io/zume/jobs/4134000002,zume,&lt;h3&gt;&lt;strong data-redactor-tag=&quot;s...
32,Data Engineer,"Seattle, WA",https://boards.greenhouse.io/zume/jobs/4057164002,zume,&lt;p&gt;&lt;strong&gt;WHO WE ARE&lt;/strong&g...
33,Data Scientist,"Seattle, WA",https://boards.greenhouse.io/zume/jobs/4055865002,zume,&lt;p&gt;&lt;strong&gt;WHO WE ARE&lt;/strong&g...


## Extract information from text descriptions
Information extracted is:

Education: Search and store highest level of education, if any, mentioned in job description.

Experience: Store a list (as a string) of all years of experience mentioned in the job description. Formats searched for in this code are "#+ years" "#years" and "#+ yrs", where # is an integer. Other patterns can be matched but only pattens that return a search for our data set were included below. 

Industry: Store the appropriate industry for each record. Industry is based on the company - external data must be loaded to match copmany to industry.

Manager: Search job title to check if is a manager position and store result as a boolean.

Skills: Search for mention of a skill and store result as a boolean in a new dataframe of size A x B where A is number of jobs/recrods and B is number of skills.


In [26]:
"""Extract Highest Level of Education"""

def educationsearch(desc):
    #input: text (job description) that is to be searched
    if any(s in desc.lower() for s in ['phd','doctorate']):
        edu = 'PhD'  
    elif any(s in desc.lower() for s in ["master's" , "graduate degree", " ms ","masters degree", "advanced degree", 'masters']):
        edu = 'MS'  
    elif any(s in desc.lower() for s in ["bachelor" , "bs ",'undergrad','degree']):
        edu = 'BS'  
    else:
        edu = 'na'
    #print edu, index
    return edu

education = pd.Series(index=data.index) 

for index, desc in (data.description).items():
    education[index] = educationsearch(desc)
    

In [27]:
"""Extract years of Experience"""
import re

experience = pd.Series(index=data.index) 
for index, desc in (data.description).items(): 
    experience[index] = str([int(x) for x in re.findall("(\d+)(?:\+ years|years|\+ yrs)",desc.lower())])
      
#since we cannot save the datatype 'list' in a dataframe, we have saved the information as a string
#to go back to a list use below code

def stringtolistofnumbers(x):
    #input a string that is obtained from str(y) where y is a list of integers such as [1] or [1,2]
    #will return the original list in an object with type list filled with integers  
    return [int(y) for y in re.findall("(\d+)",x)]
#can use stringtolistofnumbers function in analysis

In [28]:
"""Insert Industry Labels Based on Company"""
industrylabelsdata =pd.read_excel('Data Analytics Project-Industry.xlsx','Sheet1')
industrylabels = dict(zip(industrylabelsdata.Company,industrylabelsdata.Industry))
industry = pd.Series(index=data.index)
for index, record in data.company.items():
    industry[index] = industrylabels[record]
#print industry

In [29]:
"""Extract Manager Boolean"""
manager = data.title.str.contains('Manager') 

In [34]:
"""Combine all non-skill related data that was extracted from descirption into one data frame"""
information = pd.DataFrame({'experience': experience,'manager': manager,'industry': industry, 'education': education})
information

,education,experience,industry,manager
0,MS,[],Technology,False
1,na,[3],Technology,False
2,MS,[],Technology,False
3,PhD,[3],Technology,False
4,PhD,[3],Technology,False
5,BS,[],Technology,True
6,BS,[],Technology,False
7,BS,[3],Technology,False
8,BS,[],Technology,False
9,MS,[3],Technology,False


In [31]:
"""Extract Skills"""
#establish list of skills, in three categories: technologies, areas of knowledge, and soft/other skills
techskills = [' r ','python','sql','sas','excel','java','c++','matlab','tableau',\
             'hadoop','hive','presto','html','scala','oracle','postgres','mysql','php','perl','ruby','powerbi',\
              'docker', 'css', 'pyramid', 'powerpoint', 'google docs','google slides', 'jira', 'scikit-learn',\
              'javascript','spark','kafka', 'mongo','kubernetes','unix','linux','vba','aws','tensorflow','pytorch','keras',\
             'apache','d3.js']
knowledgeskills = ['statistics','analytics','modeling','algorithm design','programming','research',\
                   'software engineering', 'visualization','computer science','nosql', 'search engine optimization', \
                   'saas', 'etl','machine learning','big data']
softskills = ['communication', 'team player','negotiation','coordination', 'presentation','cross-functional','writing']
techskills.sort();knowledgeskills.sort(),softskills.sort() #alphabatize within each subset for readability

skillslist = techskills + knowledgeskills +softskills

#Fill a dataframe with True/False values based on if skill shows up in description

skillsdf=  pd.DataFrame(index=data.index, columns = skillslist).fillna(False) #initialize all fields to false
for index, desc in (data.description).items():
    for skill in skillslist:
        if skill in desc.lower():
            skillsdf[skill][index] = True

#skillsdf

#quick data summary: which skill has most occurences  
#(skillsdf==True).sum(axis=0).sort_values(ascending=False)


In [40]:
skillsdf.python

0     False
1      True
2      True
3      True
4      True
5     False
6      True
7      True
8     False
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21    False
22     True
23     True
24    False
25     True
26     True
27     True
28    False
29     True
30    False
31    False
32     True
33     True
Name: python, dtype: bool

In [41]:
pd.crosstab(information.industry,skillsdf.python)

python,False,True
industry,,
Food,2,2
Technology,6,24


In [36]:
information.industry

0     Technology 
1     Technology 
2     Technology 
3     Technology 
4     Technology 
5     Technology 
6     Technology 
7     Technology 
8     Technology 
9     Technology 
10    Technology 
11    Technology 
12    Technology 
13    Technology 
14    Technology 
15    Technology 
16    Technology 
17    Technology 
18    Technology 
19    Technology 
20    Technology 
21    Technology 
22    Technology 
23    Technology 
24    Technology 
25    Technology 
26    Technology 
27    Technology 
28    Technology 
29    Technology 
30          Food 
31          Food 
32          Food 
33          Food 
Name: industry, dtype: object

## Save and export data
Data is filtered by job title and then reindexed so each record has a unique index

In [ ]:
'''SAVE DATAFRAME OBJECTS'''
data.to_hdf('YOURFILENAMEHERE_A.h5','data', mode='w')

information.to_hdf('YOURFILENAMEHERE_B.h5','information', mode='w')

skillsdf.to_hdf('YOURFILENAMEHERE_B_V.h5','skills', mode='w')


'''SAVE TO EXCEL'''
writer=pd.ExcelWriter('YOURFILENAMEHERE.xlsx') 
data.to_excel(writer,'data')
information.to_excel(writer,'information')
skillsdf.to_excel(writer,'skills')
writer.save()



## Final Dataset is attached as "ProjectData612.xlsx

In [ ]:
"""READ DATAFRAMES"""
data= pd.read_hdf('projectdata612A.h5')
information= pd.read_hdf('projectdata612B.h5')
skills= pd.read_hdf('projectdata612C.h5')